### Ensemble Classfiers

In [162]:
import random
import numpy as np
import tensorflow as tf

from sklearn import metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from random import randint
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#### Loading the required models

In [151]:
# Loading the best face model
face_model = keras.models.load_model('best_model_face_inception.h5')
print(face_model)

# Loading the best face model
hand_model = keras.models.load_model('best_model_hand_molyswu.h5')
print(hand_model)

#### Setting up the input and output datasets

In [152]:
# Importing data
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# TODO: Need to make sure that the same image is used for both models
FACE_DATA_PATH = '/Users/preshita/Desktop/unseen_face'
HANDS_DATA_PATH = '/Users/preshita/Desktop/unseen_hand'

# Computing predictions by the best face model
face_test_generator = test_datagen.flow_from_directory(FACE_DATA_PATH,
                                                    batch_size  = 40,
                                                    class_mode  = 'categorical', 
                                                    target_size = (100, 75), shuffle = False)
face_data_inputs = face_test_generator # Face data input
y_face_preds = face_model.predict(face_test_generator) # Face data predictions

# Computing predictions by the best hand model
hand_test_generator = test_datagen.flow_from_directory(HANDS_DATA_PATH,
                                                    batch_size  = 40,
                                                    class_mode  = 'categorical', 
                                                    target_size = (100, 75), shuffle = False)
hands_data_inputs = hand_test_generator # Hand data input
y_hands_preds = hand_model.predict(hand_test_generator) # Hand data predictions

# Expected data outputs
expected_output_labels = face_test_generator.classes # Expected output labels, assuming that both models are working with the same images
data_outputs = np.zeros((expected_output_labels.size, expected_output_labels.max() + 1))
data_outputs[np.arange(expected_output_labels.size), expected_output_labels] = 1


'''
Unseen data (unextracted images)
Note: Not to be used in final model
'''
# unseen_dir = '/Users/preshita/Desktop/combined_new/unseen'
# unseen_test_generator = test_datagen.flow_from_directory(  unseen_dir,
#                                                     batch_size  = 40,
#                                                     class_mode  = 'categorical', 
#                                                     target_size = (100, 75), shuffle = False)

Found 811 images belonging to 8 classes.
21/21 [==============================] - 4s 122ms/step
Found 526 images belonging to 8 classes.
14/14 [==============================] - 8s 495ms/step


'\nUnseen data (unextracted images)\n'

#### Helper Functions

In [153]:
'''
Given an array contain the predictions from each classifier and the weights to be assigned to each classifier, this function computes the final weighted probability.
'''
def weighted_probability(num_of_classfiers, num_of_classes, networks_outputs, curr_weight_combi):
    result = [0 for i in range(0, num_of_classes)]
    sum = 0

    for i in range(0, num_of_classfiers):
        curr_network_output = networks_outputs[i]
        curr_weight = curr_weight_combi[i]

        for j in range(0, num_of_classes):
            result[j] += curr_network_output[j] * curr_weight
            sum += result[j]
    
    for k in range(0, num_of_classes): # Normalising to ensure that the final output is still in terms of probability
        result[k] = result[k] / sum

    return result

'''
Calculates the negative log loss.
'''
def fitness(y_pred, y_true): # Negative log loss function
    return metrics.log_loss(y_true, y_pred)

'''
Randomly changes a given float number (up to 2%). 
Note: The method of mutation was not stated in the research paper. 
'''
def mutate(weight_combi): #TODO: Need to double-check if this is okay
    for i in range(0, len(weight_combi)):
        weight_combi[i] = weight_combi[i] * random.uniform(0.99, 1.01)
    
    return weight_combi

'''
Given 2 different possible weight combination, this function produces a final weight combination by randomly extracting weight elements from either parent combinations.
'''
def cross_over(num_of_classifiers, parent_1, parent_2): #TODO: Need to double-check if this is okay
    cut = random.randint(0, num_of_classifiers - 1)
    new_weight_combi = parent_1[:cut] 
    new_weight_combi.extend(parent_2[cut:])

    return new_weight_combi

'''
Produces combinations of weights that can be assigned to each of the classifiers. 
'''
def generate_possible_weight_combis(num_of_classifiers, num_of_combis, weight_limit):
    possible_weight_combis = []

    while (num_of_combis > 0):
        curr_weight_combi = []
        curr_combi_len = 0

        while (curr_combi_len < num_of_classifiers):
            curr_weight = random.uniform(0, weight_limit)
            curr_weight_combi.append(curr_weight)

            curr_combi_len += 1
        
        possible_weight_combis.append(curr_weight_combi)
        num_of_combis -= 1
    
    return possible_weight_combis


#### Start of Genetic Algorithm to find the optimal weights for each classifier

In [155]:
# Defining essential variables
num_of_classifiers = 2
num_of_classes = 8
num_of_required_weight_combis = 100
weight_limit = 100
possible_weight_combis = generate_possible_weight_combis(num_of_classifiers, num_of_required_weight_combis, weight_limit)
max_num_of_iters = 5

# print(possible_weight_combis)

while (max_num_of_iters > 0):
    # Step 1: Randomly chossing 50% of the dataset to calculate the fitness scores for
    chosen_y_true = []
    chosen_y_face_pred = []
    chosen_y_hand_pred = []

    required_num_of_samples = len(data_outputs) // 2 # Rounding down

    random_indices = []
    while required_num_of_samples > 0:
        # curr_index = randint(0, len(data_outputs) - 1)
        curr_index = randint(0, 525) # TODO: Temporary work around

        if (curr_index not in random_indices):
            chosen_y_true.append(data_outputs[curr_index])
            chosen_y_face_pred.append(y_face_preds[curr_index])
            chosen_y_hand_pred.append(y_hands_preds[curr_index])

            random_indices.append(curr_index)
            required_num_of_samples -= 1

    # Step 2: Calculate the average fitness scores for each of the possible weight combinations
    fitness_and_weights = []

    for weights in possible_weight_combis:
        accumulated_fitness_score = 0
        num_of_samples = 0

        for i in range(0, len(chosen_y_true)):
            network_outputs = [chosen_y_face_pred[i], chosen_y_hand_pred[i]]
            y_pred = weighted_probability(num_of_classifiers, num_of_classes, network_outputs, weights)
            y_true = chosen_y_true[i]
            fitness_score = fitness(y_pred, y_true)
            accumulated_fitness_score += fitness_score

            num_of_samples += 1
        
        avg_fitness_score = accumulated_fitness_score / num_of_samples
        fitness_and_weights.append((avg_fitness_score, weights))
    
    # print(fitness_and_weights) # For testing

    # Step 3: Rank the weight combis from best to worse
    fitness_and_weights.sort() # The combis with the lowest log loss is at the start
    # print(fitness_and_weights) # For testing

    # Step 4: Selecting parents
    parents = []
    curr_index = 0

    # Selecting top 20% of the weight combis
    top_20_percent = int(len(fitness_and_weights) // 5) # Rounding down
    while (top_20_percent > 0):
        parents.append(fitness_and_weights[curr_index][1])
        top_20_percent -= 1
        curr_index += 1

    # Randomly choosing another 10% of the weight combinations
    another_10_percent = int(len(fitness_and_weights) // 10)  # Rounding down
    while(another_10_percent > 0):
        random_score_and_parent = random.choice(fitness_and_weights[curr_index:])
        parents.append(random_score_and_parent[1])
        fitness_and_weights.remove(random_score_and_parent)

        another_10_percent -= 1
    
    # print(parents) # For testing

    # Step 5: Randomly mutate 5% of the selected parents
    num_of_parents_to_mutate = max(1, int(len(parents) // 10))  # Rounding down
    index_of_parents_to_mutate = [random.randint(0, len(parents) - 1) for i in range(0, num_of_parents_to_mutate)]

    for index in index_of_parents_to_mutate:
        parents[index] = mutate(parents[index])
    
    # print(parents) # For testing

    # Step 6: Randomly cross over parents to produce new set of weight combinations
    new_weight_combis = []
    index_of_crossed_parents = []
    num_of_curr_weights = 0

    while (num_of_curr_weights < num_of_required_weight_combis):
        chosen_parents = (random.randint(0, len(parents) - 1), random.randint(0, len(parents) - 1))
        parent_1 = parents[chosen_parents[0]]
        parent_2 = parents[chosen_parents[1]]

        if (parent_1 != parent_2 and chosen_parents not in index_of_crossed_parents):
            new_weight_combi = cross_over(num_of_classifiers, parent_1, parent_2)
            new_weight_combis.append(new_weight_combi)
            num_of_curr_weights += 1

    possible_weight_combis = new_weight_combis
    print(possible_weight_combis) # For testing

    max_num_of_iters -= 1

# Step 7: Select the best weights combination
final_fitness_and_weights = []

for weights in possible_weight_combis:
    accumulated_fitness_score = 0
    num_of_samples = 0

    for i in range(0, len(chosen_y_true)):
            network_outputs = [chosen_y_face_pred[i], chosen_y_hand_pred[i]]
            y_pred = weighted_probability(num_of_classifiers, num_of_classes, network_outputs, weights)
            y_true = chosen_y_true[i]
            fitness_score = fitness(y_pred, y_true)
            accumulated_fitness_score += fitness_score

            num_of_samples += 1
    
    avg_fitness_score = accumulated_fitness_score / num_of_samples
    final_fitness_and_weights.append((avg_fitness_score, weights))

final_fitness_and_weights.sort() # The combis with the lowest log loss is at the start
best_weights = final_fitness_and_weights[0][1]
print("The best weight combination is: " + str(best_weights))
print("The fitness score of this combination is: " + str(final_fitness_and_weights[0][0]))

[[98.18311664413463, 20.049923510498402], [38.120301118561386, 56.76694502503822], [94.84750112714451, 24.952625895181612], [55.49718926168366, 40.0650340518719], [98.8923690433, 75.49899789322473], [46.082826324988545, 12.620305329160232], [5.625216992778126, 9.02665109316011], [31.195921271450487, 22.631018932846324], [98.8923690433, 75.49899789322473], [98.8923690433, 1.5349448441986269], [46.082826324988545, 56.76694502503822], [46.082826324988545, 56.76694502503822], [46.082826324988545, 9.118041996066719], [46.082826324988545, 40.0650340518719], [67.98899298805543, 17.4197490604741], [98.8923690433, 75.49899789322473], [98.8923690433, 75.49899789322473], [56.359675160711184, 13.691257437338821], [98.8923690433, 79.54628349516884], [58.75018831096176, 9.02665109316011], [67.98899298805543, 9.02665109316011], [43.634300882555934, 5.051245821518724], [39.9477707544503, 5.051245821518724], [95.84716748340128, 22.631018932846324], [92.69462229408612, 5.54753594800853], [98.8923690433,

#### Computing the performance of the ensemble model

In [160]:
weighted_preds = []

# for i in range(0, len(expected_output_labels)):
for i in range(0, 526): # TODO: Temporary work around
    networks_outputs = [y_face_preds[i], y_hands_preds[i]]
    weighted_preds.append(weighted_probability(num_of_classifiers, num_of_classes, networks_outputs, best_weights))

ensemble_ypred = np.argmax(weighted_preds, axis=1)

expected_output_labels = hand_test_generator.classes

# Printing out metrics
accuracy = accuracy_score(expected_output_labels, ensemble_ypred)
print('Accuracy: %f' % accuracy)

# precision tp / (tp + fp)
precision = precision_score(expected_output_labels, ensemble_ypred, average='weighted')
print('Precision: %f' % precision)

# recall: tp / (tp + fn)
recall = recall_score(expected_output_labels, ensemble_ypred, average='weighted')
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(expected_output_labels, ensemble_ypred, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.150190
Precision: 0.120289
Recall: 0.150190
F1 score: 0.117254


#### Type 3: Random forests